In [1]:
!pip install -q -U datasets

In [2]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install -q git+https://github.com/huggingface/transformers.git@main accelerate

### Loading the model and tokenizer

In [4]:
import datasets

In [5]:
from transformers import AutoTokenizer
import transformers
import torch
from tqdm import notebook
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset

import json

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
model_id = "codellama/CodeLlama-7b-Instruct-hf"

### Preparing the Pipeline

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

In [8]:
def tokenize(data):
    result = tokenizer(
        data['prompt'],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [9]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

### Generating Code

In [11]:
def llama_v2_prompt(
    messages: list[dict]
):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = f"""Provide answers in C++. Wrap around code in [CPP] and [/CPP]. Don't write any comments in generated code."""

    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages


    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]

    messages_list = [
        f"{BOS}{B_INST}{(prompt['content']).strip()}{E_INST}{(answer['content']).strip()}{EOS}"
        for prompt, answer in zip(messages[::2], messages[1::2])
    ]
    messages_list.append(f"{BOS}{B_INST}{(messages[-1]['content']).strip()}{E_INST}\n")

    return "".join(messages_list)

In [12]:
def generate_code(dataset):
    responses = []
    batch_count = 0
    batch_size = 2
    print(f"Total batches = {len(dataset)//batch_size}")
    for out in pipeline(
      KeyDataset(dataset, "prompt"),
      do_sample=True,
      temperature=0.1,
      top_p=0.9,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=768,
      batch_size = batch_size
      ):
        batch_count += 1
        for o in out:
            responses.append(o['generated_text'])
            #print(o['generated_text'])
        print(f'Processed batch: {batch_count}')
    return responses

In [13]:
def run_code_generation(dataset):
    responses = generate_code(dataset)
    return responses
  # for index, response in responses:
  #   question = row['question']
  #   code_context = row['code_context']
  #   for p in range(n_passes):
  #     prompt = f"# {question}\n{code_context}\n"
  #     instruction = {
  #       "role": "user",
  #       "content": prompt,
  #     }
  #     prompt, response = generate_code([instruction])
  #     code = extract_code(prompt, response, code_context)
  #     save_file(f"{save_path}/question-{index}-pass-{p}.py", code)

In [14]:
# extract_code(prompt, response, code_context)

In [15]:
train_df = pd.read_csv('/kaggle/input/cpp-generations-with-feedback/train_with_feedback.csv', index_col=0)
train_df.head()

,id,text,code,question,code_cleaned,functions,code_context,context,llama_prompt,response,response_code,report,feedback
0,0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""Function parameter 'a' should be pas...",Regenerate the entire code after fixing the fo...
1,1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'sumOfTwoCubes' is neve...",Regenerate the entire code after fixing the fo...
2,3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'findNthNumber' is neve...",Regenerate the entire code after fixing the fo...
3,4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'check' is never used.""...",Regenerate the entire code after fixing the fo...
4,5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""Parameter 'nums' can be declared as ...",Regenerate the entire code after fixing the fo...


In [16]:
def build_prompt(question, code_context, response_code, feedback):
    prompt = f"# {question}\n{code_context}\n"
    instruction1 = {
      "role": "user",
      "content": prompt,
    }
    response1 = {
        "role": "agent",
        "content": response_code
    }
    instruction2 = {
      "role": "user",
      "content": f"{feedback} \n{prompt}",
    }
    llama_prompt = llama_v2_prompt([instruction1, response1, instruction2])
    return llama_prompt

In [17]:
train_df['llama_prompt_with_feedback'] = train_df.apply(lambda row: build_prompt(row.question, row.code_context, row.response_code, row.feedback), axis = 1)
train_df.head()

,id,text,code,question,code_cleaned,functions,code_context,context,llama_prompt,response,response_code,report,feedback,llama_prompt_with_feedback
0,0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""Function parameter 'a' should be pas...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
1,1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'sumOfTwoCubes' is neve...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
2,3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'findNthNumber' is neve...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
3,4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'check' is never used.""...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
4,5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""Parameter 'nums' can be declared as ...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...


In [18]:
print(train_df.loc[32]['llama_prompt_with_feedback'])

<s>[INST]<<SYS>>
Provide answers in C++. Wrap around code in [CPP] and [/CPP]. Don't write any comments in generated code.
<</SYS>>

# Smallest N digit number divisible by N.
#include <iostream> 
 #include <math.h> 
 using namespace std ;
 int smallestNumber ( int N ) {[/INST]#include <iostream> 
 #include <math.h> 
 using namespace std ;
int smallestNumber(int N) {
    int i = 1;
    while (i % N!= 0) {
        i++;
    }
    return i;
}</s><s>[INST]Regenerate the entire code after fixing the following errors: 
1. The function 'smallestNumber' is never used. at line no. 4
Wrap around code in [CPP] and [/CPP]. 
# Smallest N digit number divisible by N.
#include <iostream> 
 #include <math.h> 
 using namespace std ;
 int smallestNumber ( int N ) {[/INST]



In [19]:
prompts = train_df['llama_prompt_with_feedback'].values.tolist()
prompts_dic = {'prompt': prompts}
prompts_dataset = Dataset.from_dict(prompts_dic)

In [20]:
prompts_dataset[0]

{'prompt': "<s>[INST]<<SYS>>\nProvide answers in C++. Wrap around code in [CPP] and [/CPP]. Don't write any comments in generated code.\n<</SYS>>\n\n# Maximum Prefix Sum possible by merging two given arrays.\n#include <bits/stdc++.h> \n using namespace std ;\n int maxPresum ( vector < int > a , vector < int > b ) {[/INST]#include <bits/stdc++.h> \n using namespace std ;\n int maxPresum ( vector < int > a , vector < int > b ) {\nint n = a.size();\n    int m = b.size();\n    vector < int > c ( n + m );\n    for ( int i = 0 ; i < n ; i++ ) {\n        c[i] = a[i];\n    }\n    for ( int i = 0 ; i < m ; i++ ) {\n        c[i + n] = b[i];\n    }\n    int max_sum = 0;\n    for ( int i = 0 ; i < n + m ; i++ ) {\n        if ( i == 0 ) {\n            max_sum = c[i];\n        } else {\n            max_sum = max ( max_sum, c[i] + c[i - 1] );\n        }\n    }\n    return max_sum;\n}</s><s>[INST]Regenerate the entire code after fixing the following errors: \n1. Function parameter 'a' should be passed

In [21]:
responses = run_code_generation(prompts_dataset)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Total batches = 150


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed batch: 1
Processed batch: 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Processed batch: 293
Processed batch: 294


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed batch: 295
Processed batch: 296


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed batch: 297
Processed batch: 298
Processed batch: 299
Processed batch: 300


In [22]:
len(responses)

300

In [23]:
train_df['response_feedback'] = responses
train_df.head()

,id,text,code,question,code_cleaned,functions,code_context,context,llama_prompt,response,response_code,report,feedback,llama_prompt_with_feedback,response_feedback
0,0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""Function parameter 'a' should be pas...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
1,1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'sumOfTwoCubes' is neve...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
2,3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'findNthNumber' is neve...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
3,4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""The function 'check' is never used.""...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...
4,5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...,"[{'msg': ""Parameter 'nums' can be declared as ...",Regenerate the entire code after fixing the fo...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...,<s>[INST]<<SYS>>\nProvide answers in C++. Wrap...


In [24]:
train_df.to_csv('/kaggle/working/train_response_with_feedback.csv')

### Testing on Several Queries

### Code Infilling

For Future...

In [25]:
# from transformers import pipeline
# import torch

# generator = pipeline("text-generation",model="codellama/CodeLlama-7b-hf",torch_dtype=torch.float16, device_map="auto")
# # generator('def remove_non_ascii(s: str) -> str:\n    """ <FILL_ME>\n    return result', max_new_tokens = 128, return_type = 1)

In [26]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import transformers
# import torch

# model_id = "codellama/CodeLlama-7b-hf"
# tokenizer2 = AutoTokenizer.from_pretrained(model_id)
# model2 = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16
# ).to("cuda")




In [27]:
# prompt = '''def remove_non_ascii(s: str) -> str:
#     """ <FILL_ME>
#     return result
# '''

# input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")
# output = model.generate(
#     input_ids,
#     max_new_tokens=200,
# )
# output = output[0].to("cpu")

# filling = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)


In [28]:
# print(prompt.replace("<FILL_ME>", filling))